# App Profile Recommendation

In this project we analyze the data available from the various apps available on Google Play and the App Store that our company develops to find out user engagement. As all of our apps are free, the revenue is generated trhough in-app ads. The more the user interacts with theem, the more profitable the apps become.

The main goal of the project is to find out those apps that are potentially generating most revenue. In other words, the goal is to help the developers understand what type of apps are likely to attract more users on Google Play and the App Store

# Opening and Exploring the datasets

As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.

Collecting data for over 4 million apps requires a significant amount of time and money, so we'll try to analyze a sample of the data instead. To avoid spending resources on collecting new data ourselves, we should first try to see if we can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for our goals:

A [data set](https://www.kaggle.com/lava18/google-play-store-apps) containing data about approximately 10,000 Android apps from Google Play; the data was collected in August 2018. You can download the data set directly from this [link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).

A [data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) containing data about approximately 7,000 iOS apps from the App Store; the data was collected in July 2017. You can download the data set directly from this [link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).

As we have the data available to us, we can start by opening the dataset and then explore it.

In [1]:
from csv import reader
opened_file_google = open('dataset\googleplaystore.csv', encoding="utf8")
read_file_google = reader(opened_file_google)
apps_data_google = list(read_file_google)
google_header = apps_data_google[0]
google_data = apps_data_google[1:]

opened_file_apple = open('dataset\AppleStore.csv', encoding="utf8")
read_file_apple = reader(opened_file_apple)
apps_data_apple = list(read_file_apple)
apple_header = apps_data_apple[0]
apple_data = apps_data_apple[1:]

To make the dataset easily explorable, a function `explore()` is created below. It takes as input the dataset to be explored along with the start row and end row for the exploration. It can also provide us with the information such as the total number of rows and columns the dataset has. This function can be used to explore the data repeatedly with great readability.

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

We are ready to use the `explore()` function to start exploring the data. Below we printed out first 5 rows for each dataset. First, the Google store data and then the Apple play store data. 

As can be seen from below, Google play store dataset has total **10842** rows including the header row (that has all the column names) and **13** columns in total

In [3]:
explore_data(google_data, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


On the other hand, Apple store dataset has **7198** rows including the header row and a grand total of **16** columns.

In [4]:
explore_data(apple_data, 0, 3, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16


Let us now take a look at the column names (header row) for each dataset separately. 

Firstly, we look at the column names of the Google play store dataset. A detailed description of the columns are given in the [documentation](https://www.kaggle.com/lava18/google-play-store-apps).

Some of the useful columns for our analysis can be `'App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Content Rating', 'Genres'`.

In [5]:
print(google_header)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


Now let us take a glance at the column names of the Apple store dataset. A detailed description of the columns are given in the [documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps). 

From these set of columns, it looks like `'track_name', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'prime_genre'` are some of the columns that can be useful for our analysis.

In [6]:
print(apple_header)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


# Deleting wrong data

The Google Play data set has a dedicated [discussion section](https://www.kaggle.com/lava18/google-play-store-apps/discussion), and we can see that [one of the discussions](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) describes an error for a certain row.

The row in question is `google_data[10472]`. According to the discussion this entry has missing `'Rating'` and a column shift happened for next columns. Let's first print out this row in conjunction with the header row and another valid row so that we can better identify the wrongness of the entry.

In [7]:
print(google_header, '\n\n', google_data[10472], '\n\n', google_data[0])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

 ['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up'] 

 ['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


After further analyzing the **10472** entry, it looks like the data for the `'Category'` column is missing and for that reason all the data in the row shifted one column to the left. Clearly this entry is wrong and we will delete this entry below.

In [8]:
# deleting wrong entry for 10472

del google_data[10472]

Let's print and verify whether that entry was truly deleted from the dataset

In [9]:
print(google_data[10472])

['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


# Removing Duplicate Data

Now, we are going to look for duplicate data/ entries (multiple entry for the same app) in both the datasets. If we find duplicate data/entries, we are going to remove all of them keeping only the most recent entry. 

**First, we take a look at the *Google play store* dataset.** We can decide on which entry is the most recent depending on the count of the user reviews which can be found on the column: `'Reviews'`. Amongst all the duplicate entries, the entry that has the maximum number of user reviews, we can determine that as the most recent entry for that app and use that for our analysis.

We can write a function `detect_uniques_and_duplicates()` that will detect duplicate entries for an app for a given dataset and store the duplicate apps' names as a list named `duplicate_entries`. It will also store the unique apps' names as another list named `unique_entries`. This function will also take a column index of the dataset based on which the duplicity will be checked in the dataset.

Finally, we will get both lists (`duplicate_entries`, `unique_entries`) as output from the function `detect_uniques_and_duplicates()`.

In [10]:
def detect_uniques_and_duplicates(dataSet, col):
    unique_entries = []
    duplicate_entries = []
    
    for each_row in dataSet:
        if each_row[col] in unique_entries:
            duplicate_entries.append(each_row[col])
        else:
            unique_entries.append(each_row[col])
    
    return duplicate_entries, unique_entries

Below, we use `detect_uniques_and_duplicates()` function on Google Play store dataset and find out that there are **1181** duplicate data in it. Note, that, as a second parameter/ input to the function, `0` is used. Because, the column `'App'` which is the first column (index `0`) for each row in the dataset is the column that's used to check for duplicity. 

Some of the duplicate apps' names in the Google Play store dataset are also shown below.

In [11]:
dupDataGoogle, uniqueDataGoogle = detect_uniques_and_duplicates(google_data, 0)
print("Total number of duplicate entries in the Google play store dataset: " + str(len(dupDataGoogle)))
print("\n")
print("Some of the apps that has duplicate entries in the Google play store dataset: ")
print("\n")
print(dupDataGoogle[:13])

Total number of duplicate entries in the Google play store dataset: 1181


Some of the apps that has duplicate entries in the Google play store dataset: 


['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses']


As we have found **1181** duplicate entries, we can find out what the total number of unique entries should be by removing it from the total data count. As can be seen below, the total number of unique entries should be **9659**.

In [12]:
expected_length = len(google_data)-len(dupDataGoogle)
print("Expected unique entries: ", expected_length)

Expected unique entries:  9659


To remove the duplicates, we will create a dictionary `reviews_max`, where each dictionary key is a unique app name and the corresponding dictionary value is the highest number of reviews of that app. Therefore, the length of this dictionary should **9659** as this is the number of expected unique entries. We can verify that by printing the length of `reviews_max`.

In [13]:
reviews_max = {}

for each_row in google_data:
    name = each_row[0]
    n_reviews = float(each_row[3]) # 3 is the column index for 'Reviews' column in the dataset
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    if name not in reviews_max:
        reviews_max[name] = n_reviews

print("Expected Length: ", expected_length)
print("Actual Length: ", len(reviews_max))

Expected Length:  9659
Actual Length:  9659


Now, we can use the information stored in the dictionary `reviews_max` and create a new data set `android_clean`, which will have only one entry per app (and for each app, we'll only select the entry with the highest number of reviews). In the code cell below:

- We start by initializing two empty lists, `android_clean` and `already_added`.
- We loop through the `google_data` data set, and for every iteration:
    - We isolate the name of the app and the number of reviews.
    - We add the current row (`each_row`) to the `android_clean` list, and the app name (`name`) to the `already_added` list if:
        - The number of reviews of the current app matches the number of reviews of that app as described in the `reviews_max` dictionary; and
        - The name of the app is not already in the `already_added` list. We need to add this supplementary condition to account for those cases where the highest number of reviews of a duplicate app is the same for more than one entry (for example, the Box app has three entries, and the number of reviews is the same). If we just check for `reviews_max[name] == n_reviews`, we'll still end up with duplicate entries for some apps.

In [14]:
android_clean = []
already_added = []

for each_row in google_data:
    name = each_row[0]
    n_reviews = float(each_row[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(each_row)
        already_added.append(name)

Finally, we can verify the length (number of rows) of `android_clean` as it should be the same as the length of `reviews_max` which is **9659** by exploring the dataset using `explore_data()` function below.

In [15]:
explore_data(android_clean, 0, 5, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


Number of rows: 9659
Number of columns: 13
